In [73]:
import re as r
import pandas as pd
from pymongo import MongoClient # I'm importing my labrary
client = MongoClient("localhost:27017") # I'm giving my local adress
db = client["Ironhack"] # name of the database
collection = db.get_collection("companies") # name of the collection

In [74]:
# The offices are insisde another diccionary, so we are going to build 2 different searchs and then append them.
# The estimated Wroth of an startup is a 15% per year so lets find companies that actually has the size of our company:
o_size = 88
tasa = 0.15
tiempo = 5
final_size = o_size * (1 + tasa) ** tiempo


# 1- Search A (Employees, company name)

# # Now we can filter for our parameters, our company has 
filter = {"category_code": "games_video", "number_of_employees" : {'$gte': o_size, '$lt': final_size},"offices.country_code": "USA"}
projection = {"_id": 0, "name": 1, "number_of_employees":1}
results = list(collection.find(filter, projection).sort("number_of_employees", -1).limit(30))
df = pd.DataFrame(results)

# 2- Search A (city of the ofice)

filter = {"category_code": "games_video", "number_of_employees" : {'$gte': o_size, '$lt': final_size},"offices.country_code": "USA"}
projection = {"_id": 0, "offices.city":1}
results = list(collection.find(filter, projection).sort("number_of_employees", -1).limit(30))
df_office = pd.DataFrame(results)
df_office = pd.DataFrame(df_office['offices'].tolist())
df_office



# 3  - I merge the 2 tables.

df = pd.merge(df, df_office, left_index=True, right_index=True)
pattern1 = r"^\{'city': '"
pattern2 = r"'\}$"
for col in df.columns:
    df[col] = df[col].astype(str).str.replace(pattern1, '', regex=True)
for col in df.columns:
    df[col] = df[col].astype(str).str.replace(pattern2, '', regex=True)
for col in df.columns:
    df[col] = df[col].astype(str).str.replace("None", 'n', regex=True)
df = df.drop(2, axis=1)
df = df.drop(3, axis=1)
header_mapping = {
    'name': 'Name',
    'number_of_employees': 'Employees',    
    0: 'Office_1',
    1: 'Office_2',
}
df = df.rename(columns=header_mapping)
df

,Name,Employees,Office_1,Office_2
0,hulu,153,Los Angeles,n
1,Boonty,150,New York,n
2,ngmoco,150,San Francisco,n
3,i-Jet Media,150,Sunnyvale,Ekaterinburg
4,NaturalMotion,150,Oxford,San Francisco
5,Z2Live,130,Seattle,n
6,RGB Networks,125,Sunnyvale,n
7,Modern Feed,123,Los Angeles,n
8,Livestream,120,New York,n
9,Zynga,115,San Francisco,


In [75]:
counts = df['Office_1'].value_counts().reset_index()

# Renombrar las columnas del nuevo DataFrame
counts.columns = ['city', 'repetition']

# Imprimir el nuevo DataFrame con el recuento de valores únicos en 'city'
counts

,city,repetition
0,San Francisco,4
1,Los Angeles,3
2,New York,3
3,Sunnyvale,3
4,Oxford,1
5,Seattle,1
6,Kharkov,1
7,Redwood City,1
8,Brooklyn,1


In [82]:
# # Now we can filter for our parameters, our company has 
filter = {"category_code": "games_video", "number_of_employees" : {'$gte': o_size, '$lt': final_size},"offices.country_code": "USA", "offices.city":"San Francisco", "name": "ngmoco"}
projection = {"_id": 0, "name": 1, "offices.latitude":1, "offices.longitude":1}
results = list(collection.find(filter, projection))
df_city = pd.DataFrame(results)
df_city = pd.json_normalize(df_city['offices'])
df_city = pd.DataFrame(df_city[0].tolist())
df_city

,latitude,longitude
0,37.77051,-122.401581


In [81]:
new_df = pd.json_normalize(df_city['offices'])
new_df = pd.DataFrame(new_df[0].tolist())
new_df

,latitude,longitude
0,37.77051,-122.401581
